<a href="https://colab.research.google.com/github/sinhajiya/NLP/blob/main/NLP_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import bigrams, trigrams
import re

In [2]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

# Question 1

## Cloning GitHub repo for dataset

In [3]:
!git clone https://github.com/islnlp/Assignment_1_2025

Cloning into 'Assignment_1_2025'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 35 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 1.06 MiB | 7.64 MiB/s, done.
Resolving deltas: 100% (6/6), done.


## Loading data and preprocessing

In [40]:
def load_data(name):
  root_fp = f"/content/Assignment_1_2025/{name}"
  train = pd.read_csv(os.path.join(root_fp, "train.csv"))
  val = pd.read_csv(os.path.join(root_fp, "val.csv"))
  train = train.dropna(subset=['Sentence'])
  val = val.dropna(subset=['Sentence'])
  return train, val

In [41]:
def preprocess_text(Sentence):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    Sentence = Sentence.lower() # All lower case characters
    Sentence = re.sub(url_pattern, "", Sentence)  # Remove all URLs
    Sentence = re.sub(r"\.{2,}", ".", Sentence)  # Replace multiple dots with a single dot
    Sentence = re.sub(r"\s+", " ", Sentence).strip()  # Remove extra spaces
    return Sentence

In [48]:
def load_and_preprocess_data(name):
  train, val = load_data(name)
  print(f"Loading and preprocessing the {name} data... \n")
  print(f"Train shape: {train.shape}, Val shape: {val.shape}\n")
  print(f"Train head: \n {train.head()}\n")
  print(f"Val head: \n {val.head()}\n")
  print(f"The count of the tags in training data is \n {train['Tag'].value_counts()}\n")
  print(f"The count of the tags in validation data is \n {val['Tag'].value_counts()}\n")
  train["Sentence_preprocessed"] = train["Sentence"].astype(str).apply(preprocess_text)
  val["Sentence_preprocessed"] = val["Sentence"].astype(str).apply(preprocess_text)
  print(f"Before preprocessing: \n")
  print(f"{train['Sentence'][21]} \n {train['Sentence'][27]}")
  print(f"After preprocessing: \n")
  print(f"{train['Sentence_preprocessed'][21]} \n {train['Sentence_preprocessed'][27]}")

  return train, val

In [49]:
hate_train,hate_val = load_and_preprocess_data("hate")

Loading and preprocessing the hate data... 

Train shape: (3660, 2), Val shape: (457, 2)

Train head: 
                                             Sentence  Tag
0  #hariyana mey ek week mey teen bachchiyo ke Sa...    1
1  indira Gandhi ko marne wala sikh.Rajiv Gandhi ...    1
2  ishliye corruption ke jariye sab ki khoon choo...    1
3  Pakistaniyon ko aisi news Maamul sa ho Gaya ha...    0
4  Apne national anthem ko change karo and yeh li...    1

Val head: 
                                             Sentence  Tag
0          Sahee bolay ho kal Pakistani ka rape hoga    0
1                    rape. Pyaar rape se kum nhi hai    0
2  ye log mandir me hi q jakar rape krte kaya bi ...    0
3  Delhi-Dehradun train mein 1seat par baithne k ...    0
4  #GadhaAkallesh ne Kairana/Sahibabad/Buxar se H...    1

The count of the tags in training data is 
 Tag
0    2307
1    1353
Name: count, dtype: int64

The count of the tags in validation data is 
 Tag
0    309
1    148
Name: count, dtype: int

## Feature Extraction

Using N-gram language models (unigrams, bigrams, trigrams)
as features.